In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

dataset = pd.read_csv('kddcup.data_10_percent_corrected')


2022-02-18 13:54:24.992694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-18 13:54:24.992748: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# 처음 다섯 개 데이터를 조회
dataset.head()

,0,tcp,http,SF,181,5450,0.1,0.2,0.3,0.4,...,9.1,1.00.1,0.00.6,0.11.1,0.00.7,0.00.8,0.00.9,0.00.10,0.00.11,normal.
0,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [3]:
#  마지막 label 컬럼 안에 담긴 고유의 카테고리 값을 확인
dataset['normal.'].unique()

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

In [4]:
# normal. 을 제외한 모든 카테고리를 attack.으로 변경
dataset['normal.'] = dataset['normal.'].replace(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'], 'attack')
dataset['normal.'].unique()

array(['normal.', 'attack'], dtype=object)

In [5]:
# x = 마지막 label 열을 제외한 모든 열을 특징으로 사용
# y = 마지막 label 열을 카테고리로 사용

x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 41].values

print(x.shape, y.shape)

(494020, 41) (494020,)


In [6]:
uniq1 = dataset.tcp.unique()
uniq2 = dataset.http.unique()
uniq3 = dataset.SF.unique()

print(uniq1, '\n', uniq2, '\n', uniq3)

['tcp' 'udp' 'icmp'] 
 ['http' 'smtp' 'finger' 'domain_u' 'auth' 'telnet' 'ftp' 'eco_i' 'ntp_u'
 'ecr_i' 'other' 'private' 'pop_3' 'ftp_data' 'rje' 'time' 'mtp' 'link'
 'remote_job' 'gopher' 'ssh' 'name' 'whois' 'domain' 'login' 'imap4'
 'daytime' 'ctf' 'nntp' 'shell' 'IRC' 'nnsp' 'http_443' 'exec' 'printer'
 'efs' 'courier' 'uucp' 'klogin' 'kshell' 'echo' 'discard' 'systat'
 'supdup' 'iso_tsap' 'hostnames' 'csnet_ns' 'pop_2' 'sunrpc' 'uucp_path'
 'netbios_ns' 'netbios_ssn' 'netbios_dgm' 'sql_net' 'vmnet' 'bgp' 'Z39_50'
 'ldap' 'netstat' 'urh_i' 'X11' 'urp_i' 'pm_dump' 'tftp_u' 'tim_i' 'red_i'] 
 ['SF' 'S1' 'REJ' 'S2' 'S0' 'S3' 'RSTO' 'RSTR' 'RSTOS0' 'OTH' 'SH']


In [7]:
'''
# tcp(프로토콜), http(서비스), SF(플래그) 열을 One-Hot Encoding 변환
labelencoder_x_1 = LabelEncoder()
labelencoder_x_2 = LabelEncoder()
labelencoder_x_3 = LabelEncoder()

x[:, 1] = labelencoder_x_1.fit_transform(x[:, 1])
x[:, 2] = labelencoder_x_2.fit_transform(x[:, 2])
x[:, 3] = labelencoder_x_3.fit_transform(x[:, 3])

onehotencoder_1 = OneHotEncoder(categorical_features = [1])
onehotencoder_2 = OneHotEncoder(categorical_features = [4])  # + 3
onehotencoder_3 = OneHotEncoder(categorical_features = [70]) # + 66

x = onehotencoder_1.fit_transform(x).toarray()
x = onehotencoder_2.fit_transform(x).toarray()
x = onehotencoder_3.fit_transform(x).toarray()

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
'''
'''
from sklearn.compose import ColumnTransformer
# TCP(프로토콜)
ct = ColumnTransformer([("tcp", OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 1] = labelencoder_x.fit_transform(x[:, 1])

# HTTP(프로토콜)
ct = ColumnTransformer([("http", OneHotEncoder(), [4])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 2] = labelencoder_x.fit_transform(x[:, 2])

# SF(프로토콜)
ct = ColumnTransformer([("sf", OneHotEncoder(), [70])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 3] = labelencoder_x.fit_transform(x[:, 3])

print(x.shape, y.shape)
'''

'\nfrom sklearn.compose import ColumnTransformer\n# TCP(프로토콜)\nct = ColumnTransformer([("tcp", OneHotEncoder(), [1])], remainder = \'passthrough\')\nx = ct.fit_transform(x)\nlabelencoder_x = LabelEncoder()\nx[:, 1] = labelencoder_x.fit_transform(x[:, 1])\n\n# HTTP(프로토콜)\nct = ColumnTransformer([("http", OneHotEncoder(), [4])], remainder = \'passthrough\')\nx = ct.fit_transform(x)\nlabelencoder_x = LabelEncoder()\nx[:, 2] = labelencoder_x.fit_transform(x[:, 2])\n\n# SF(프로토콜)\nct = ColumnTransformer([("sf", OneHotEncoder(), [70])], remainder = \'passthrough\')\nx = ct.fit_transform(x)\nlabelencoder_x = LabelEncoder()\nx[:, 3] = labelencoder_x.fit_transform(x[:, 3])\n\nprint(x.shape, y.shape)\n'

In [18]:
# tcp(프로토콜), http(서비스), SF(플래그) 열을 One-Hot Encoding 변환
labelencoder_x_1 = LabelEncoder()
labelencoder_x_2 = LabelEncoder()
labelencoder_x_3 = LabelEncoder()

x[:, 1] = labelencoder_x_1.fit_transform(x[:, 1])
x[:, 2] = labelencoder_x_2.fit_transform(x[:, 2])
x[:, 3] = labelencoder_x_3.fit_transform(x[:, 3])

onehotencoder_1 = OneHotEncoder(categories=x[1])
onehotencoder_2 = OneHotEncoder(categories=x[4])  # + 3
onehotencoder_3 = OneHotEncoder(categories=x[70]) # + 66

x = onehotencoder_1.fit_transform(x.any()).toarray()
x = onehotencoder_2.fit_transform(x[:,4].reshape(-1,1)).toarray()
x = onehotencoder_3.fit_transform(x[:,70].reshape(-1,1)).toarray()

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
'''
'''
from sklearn.compose import ColumnTransformer
# TCP(프로토콜)
ct = ColumnTransformer([("tcp", OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 1] = labelencoder_x.fit_transform(x[:, 1])

# HTTP(프로토콜)
ct = ColumnTransformer([("http", OneHotEncoder(), [4])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 2] = labelencoder_x.fit_transform(x[:, 2])

# SF(프로토콜)
ct = ColumnTransformer([("sf", OneHotEncoder(), [70])], remainder = 'passthrough')
x = ct.fit_transform(x)
labelencoder_x = LabelEncoder()
x[:, 3] = labelencoder_x.fit_transform(x[:, 3])

print(x.shape, y.shape)

# 최종 데이터 개수
# X = 494,020 x 118
# Y = 494,020 x 1

ValueError: Expected 2D array, got scalar array instead:
array=1.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# X와 Y를 훈련 데이터(train)와 테스트 데이터(test)로 분류 = 70 : 30
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_oh, y, test_size = 0.3, random_state = 0)

# 가우시안 기반 나이브 베이즈 알고리즘 초기화 및 학습(fit)
classifier = GaussianNB()
classifier.fit(x_train, y_train)

# 학습한 모델에 테스트 데이터를 넣어 결과를 확인(predict)
y_pred = classifier.predict(x_test)

# 혼돈 행렬과 정확도, F1 점수로 모델 평가
cnf_matrix = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy : ", acc, "/ F1-Score : ", f1)
print(cnf_matrix)